In [ ]:
# THis IS CNN USING DL   this is correct to used to DL correct accoracy

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.utils import class_weight
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Your data loading and merging code for CNN
file_paths = ['/content/merged_file.xlsx']
merged_data_cnn = pd.DataFrame()

for file_path in file_paths:
    data = pd.read_excel(file_path)
    merged_data_cnn = merged_data_cnn.append(data, ignore_index=True)

# Check if the specified columns exist in the DataFrame
features_column_cnn = 'Text Content'
target_column_cnn = 'Category'
if set([features_column_cnn, target_column_cnn]).issubset(merged_data_cnn.columns):
    features_cnn = merged_data_cnn[features_column_cnn]
    target_cnn = merged_data_cnn[target_column_cnn]

    label_encoder_cnn = LabelEncoder()
    y_numerical_cnn = label_encoder_cnn.fit_transform(target_cnn)

    tokenizer_cnn = Tokenizer()
    tokenizer_cnn.fit_on_texts(features_cnn)
    total_words_cnn = len(tokenizer_cnn.word_index) + 1

    input_sequences_cnn = tokenizer_cnn.texts_to_sequences(features_cnn)

    max_sequence_length_cnn = max(len(seq) for seq in input_sequences_cnn)
    padded_sequences_cnn = pad_sequences(input_sequences_cnn, maxlen=max_sequence_length_cnn, padding='post')

    X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(padded_sequences_cnn, y_numerical_cnn, test_size=0.2, random_state=42)

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_cnn), y=y_train_cnn)
class_weights_dict = dict(zip(np.unique(y_train_cnn), class_weights))

# Build and compile the CNN model with class weights
learning_rate = 0.00001  # You can adjust this value as needed
optimizer_cnn = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model_cnn = Sequential()
model_cnn.add(Embedding(input_dim=total_words_cnn, output_dim=32, input_length=max_sequence_length_cnn))
model_cnn.add(Conv1D(128, 5, activation='relu'))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(64, activation='relu'))
model_cnn.add(Dense(len(label_encoder_cnn.classes_), activation='softmax'))

model_cnn.compile(optimizer=optimizer_cnn, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN model with class weights
history = model_cnn.fit(X_train_cnn, y_train_cnn, epochs=30, batch_size=32, validation_split=0.2, class_weight=class_weights_dict)



# Save the trained CNN model
model_cnn.save('spell_checker_CNN_model.h5')

# Evaluate the CNN model and make predictions
loss_cnn, accuracy_cnn = model_cnn.evaluate(X_test_cnn, y_test_cnn)
print(f'CNN Model Test Loss: {loss_cnn:.4f}, Test Accuracy: {accuracy_cnn:.4f}')

# Make predictions on the test set using the CNN model
y_pred_cnn = model_cnn.predict(X_test_cnn)
y_pred_classes_cnn = np.argmax(y_pred_cnn, axis=1)

# Confusion matrix
cm_cnn = confusion_matrix(y_test_cnn, y_pred_classes_cnn)
labels_cnn = sorted(set(list(y_test_cnn) + list(y_pred_classes_cnn)))
plt.figure(figsize=(8, 6))
ax = plt.gca()
sns.heatmap(cm_cnn, annot=True, ax=ax, cmap='Blues')
ax.set_title('Confusion Matrix')
ax.set_xlabel('Predicted Category')
ax.set_ylabel('Actual Category')
ax.set_xticks(np.arange(len(labels_cnn)) + 0.5, labels=labels_cnn, rotation=45)
ax.set_yticks(np.arange(len(labels_cnn)) + 0.5, labels=labels_cnn)
plt.show()

# Calculate class-specific accuracies for the CNN model
class_accuracies_cnn = cm_cnn.diagonal() / cm_cnn.sum(axis=1)
for i, acc in enumerate(class_accuracies_cnn):
    print(f'Accuracy for class {label_encoder_cnn.classes_[i]}: {acc:.2%}')

# Print classification report
classification_rep = classification_report(y_test_cnn, y_pred_classes_cnn, target_names=label_encoder_cnn.classes_)
print(classification_rep)

# Create a DataFrame to store results
results_df = pd.DataFrame({
    'Accuracy': [accuracy_cnn],
    'Loss': [loss_cnn],
    'Confusion Matrix': [cm_cnn],
    'Class Accuracies': [class_accuracies_cnn],
    'Classification Report': [classification_rep]
})

# Save the DataFrame to an Excel file
excel_file_name = "CNN_results.xlsx"
results_df.to_excel(excel_file_name, index=False)
print(f"Results saved to {excel_file_name}")


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Plot the training and validation loss
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss for CNN Model')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Save the plot as an image
plt.savefig('/content/CNN_results_Loss.png')

# Create a DataFrame with loss values
loss_data = pd.DataFrame({'Epochs': epochs, 'Training Loss': loss, 'Validation Loss': val_loss})

# Save the loss values to an Excel file
loss_data.to_excel('/content/CNN_results.xlsx', index=False)

# Display the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy for CNN model')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Save the plot as an image
plt.savefig('/content/CNN_results_accuracy.png')

# Create a DataFrame with accuracy values
accuracy_data = pd.DataFrame({'Epochs': epochs, 'Training Accuracy': acc, 'Validation Accuracy': val_acc})

# Save the accuracy values to an Excel file
accuracy_data.to_excel('/content/CNN_results.xlsx', index=False)

# Display the plot
plt.show()


In [ ]:
# THis IS RNN USING DL   this is correct to used to DL correct accoracy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.utils import class_weight
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Your data loading and merging code for RNN
file_paths = ['/content/merged_file.xlsx']

merged_data_rnn = pd.DataFrame()

for file_path in file_paths:
    data = pd.read_excel(file_path)
    merged_data_rnn = merged_data_rnn.append(data, ignore_index=True)

# Check if the specified columns exist in the DataFrame
features_column_rnn = 'Text Content'
target_column_rnn = 'Category'
if set([features_column_rnn, target_column_rnn]).issubset(merged_data_rnn.columns):
    features_rnn = merged_data_rnn[features_column_rnn]
    target_rnn = merged_data_rnn[target_column_rnn]

    label_encoder_rnn = LabelEncoder()
    y_numerical_rnn = label_encoder_rnn.fit_transform(target_rnn)

    tokenizer_rnn = Tokenizer()
    tokenizer_rnn.fit_on_texts(features_rnn)
    total_words_rnn = len(tokenizer_rnn.word_index) + 1

    input_sequences_rnn = tokenizer_rnn.texts_to_sequences(features_rnn)

    max_sequence_length_rnn = max(len(seq) for seq in input_sequences_rnn)
    padded_sequences_rnn = pad_sequences(input_sequences_rnn, maxlen=max_sequence_length_rnn, padding='post')

    X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(padded_sequences_rnn, y_numerical_rnn, test_size=0.2, random_state=42)

# Calculate class weights
class_weights_rnn = compute_class_weight('balanced', classes=np.unique(y_train_rnn), y=y_train_rnn)
class_weights_dict_rnn = dict(zip(np.unique(y_train_rnn), class_weights_rnn))

# Build and compile the RNN model with class weights
learning_rate_rnn = 0.00001  # You can adjust this value as needed
optimizer_rnn = tf.keras.optimizers.Adam(learning_rate=learning_rate_rnn)

model_rnn = Sequential()
model_rnn.add(Embedding(input_dim=total_words_rnn, output_dim=32, input_length=max_sequence_length_rnn))
model_rnn.add(SimpleRNN(128))  # Replace 'LSTM' with 'SimpleRNN'
model_rnn.add(Dense(64, activation='relu'))
model_rnn.add(Dense(len(label_encoder_rnn.classes_), activation='softmax'))


model_rnn.compile(optimizer=optimizer_rnn, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the RNN model with class weights
model_rnn.fit(X_train_rnn, y_train_rnn, epochs=30, batch_size=32, validation_split=0.2, class_weight=class_weights_dict_rnn)

# Save the trained CNN model
model_rnn.save('spell_checker_RNN_model.h5')

# Evaluate the RNN model and make predictions
loss_rnn, accuracy_rnn = model_rnn.evaluate(X_test_rnn, y_test_rnn)
print(f'RNN Model Test Loss: {loss_rnn:.4f}, Test Accuracy: {accuracy_rnn:.4f}')

# Make predictions on the test set using the RNN model
y_pred_rnn = model_rnn.predict(X_test_rnn)
y_pred_classes_rnn = np.argmax(y_pred_rnn, axis=1)

# Confusion matrix
cm_rnn = confusion_matrix(y_test_rnn, y_pred_classes_rnn)
labels_rnn = sorted(set(list(y_test_rnn) + list(y_pred_classes_rnn)))
plt.figure(figsize=(8, 6))
ax_rnn = plt.gca()
sns.heatmap(cm_rnn, annot=True, ax=ax_rnn, cmap='Blues')
ax_rnn.set_title('Confusion Matrix')
ax_rnn.set_xlabel('Predicted Category')
ax_rnn.set_ylabel('Actual Category')
ax_rnn.set_xticks(np.arange(len(labels_rnn)) + 0.5, labels=labels_rnn, rotation=45)
ax_rnn.set_yticks(np.arange(len(labels_rnn)) + 0.5, labels=labels_rnn)
plt.show()

# Calculate class-specific accuracies for the RNN model
class_accuracies_rnn = cm_rnn.diagonal() / cm_rnn.sum(axis=1)
for i, acc in enumerate(class_accuracies_rnn):
    print(f'Accuracy for class {label_encoder_rnn.classes_[i]}: {acc:.2%}')

# Print classification report
print(classification_report(y_test_rnn, y_pred_classes_rnn, target_names=label_encoder_rnn.classes_))


# Create a DataFrame to store results for RNN
results_df_rnn = pd.DataFrame({
    'Accuracy': [accuracy_rnn],
    'Loss': [loss_rnn],
    'Confusion Matrix': [cm_rnn],
    'Class Accuracies': [class_accuracies_rnn],
    'Classification Report': [classification_report(y_test_rnn, y_pred_classes_rnn, target_names=label_encoder_rnn.classes_)]
})

# Save the DataFrame to an Excel file for RNN
excel_file_name_rnn = "RNN_results.xlsx"
results_df_rnn.to_excel(excel_file_name_rnn, index=False)
print(f"RNN Results saved to {excel_file_name_rnn}")



In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Plot the training and validation loss
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss for RNN model')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Save the plot as an image
plt.savefig('/content/RNN_results_Loss.png')

# Create a DataFrame with loss values
loss_data = pd.DataFrame({'Epochs': epochs, 'Training Loss': loss, 'Validation Loss': val_loss})

# Save the loss values to an Excel file
loss_data.to_excel('/content/RNN_results.xlsx', index=False)

# Display the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy for RNN model')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Save the plot as an image
plt.savefig('/content/RNN_results_accuracy.png')

# Create a DataFrame with accuracy values
accuracy_data = pd.DataFrame({'Epochs': epochs, 'Training Accuracy': acc, 'Validation Accuracy': val_acc})

# Save the accuracy values to an Excel file
accuracy_data.to_excel('/content/RNN_results.xlsx', index=False)

# Display the plot
plt.show()


In [ ]:
# THis IS LSTM USING DL   this is correct to used to DL correct accoracy

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.utils import class_weight
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Your data loading and merging code for LSTM
file_paths = ['/content/Sachal_jo_kalam_Scraping_data.xlsx', '/content/SHAH_jo_risalo_Scraping_data.xlsx', '/content/Ayaz_shaikh_Scraping_data.xlsx', '/content/Altaf_shaikh_Scraping_data.xlsx']

merged_data_lstm = pd.DataFrame()

for file_path in file_paths:
    data = pd.read_excel(file_path)
    merged_data_lstm = merged_data_lstm.append(data, ignore_index=True)

# Check if the specified columns exist in the DataFrame
features_column_lstm = 'Text Content'
target_column_lstm = 'Category'
if set([features_column_lstm, target_column_lstm]).issubset(merged_data_lstm.columns):
    features_lstm = merged_data_lstm[features_column_lstm]
    target_lstm = merged_data_lstm[target_column_lstm]

    label_encoder_lstm = LabelEncoder()
    y_numerical_lstm = label_encoder_lstm.fit_transform(target_lstm)

    tokenizer_lstm = Tokenizer()
    tokenizer_lstm.fit_on_texts(features_lstm)
    total_words_lstm = len(tokenizer_lstm.word_index) + 1

    input_sequences_lstm = tokenizer_lstm.texts_to_sequences(features_lstm)

    max_sequence_length_lstm = max(len(seq) for seq in input_sequences_lstm)
    padded_sequences_lstm = pad_sequences(input_sequences_lstm, maxlen=max_sequence_length_lstm, padding='post')

    X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(padded_sequences_lstm, y_numerical_lstm, test_size=0.2, random_state=42)

# Calculate class weights
class_weights_lstm = compute_class_weight('balanced', classes=np.unique(y_train_lstm), y=y_train_lstm)
class_weights_dict_lstm = dict(zip(np.unique(y_train_lstm), class_weights_lstm))

# Build and compile the LSTM model with class weights
learning_rate_lstm = 0.00001  # You can adjust this value as needed
optimizer_lstm = tf.keras.optimizers.Adam(learning_rate=learning_rate_lstm)

model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=total_words_lstm, output_dim=32, input_length=max_sequence_length_lstm))
model_lstm.add(LSTM(128))  # Replace 'SimpleRNN' with 'LSTM'
model_lstm.add(Dense(64, activation='relu'))
model_lstm.add(Dense(len(label_encoder_lstm.classes_), activation='softmax'))

model_lstm.compile(optimizer=optimizer_lstm, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the LSTM model with class weights
model_lstm.fit(X_train_lstm, y_train_lstm, epochs=30, batch_size=32, validation_split=0.2, class_weight=class_weights_dict_lstm)

# Save the trained CNN model
model_lstm.save('spell_checker_LSTM_model.h5')

# Evaluate the LSTM model and make predictions
loss_lstm, accuracy_lstm = model_lstm.evaluate(X_test_lstm, y_test_lstm)
print(f'LSTM Model Test Loss: {loss_lstm:.4f}, Test Accuracy: {accuracy_lstm:.4f}')

# Make predictions on the test set using the LSTM model
y_pred_lstm = model_lstm.predict(X_test_lstm)
y_pred_classes_lstm = np.argmax(y_pred_lstm, axis=1)

# Confusion matrix
cm_lstm = confusion_matrix(y_test_lstm, y_pred_classes_lstm)
labels_lstm = sorted(set(list(y_test_lstm) + list(y_pred_classes_lstm)))
plt.figure(figsize=(8, 6))
ax_lstm = plt.gca()
sns.heatmap(cm_lstm, annot=True, ax=ax_lstm, cmap='Blues')
ax_lstm.set_title('Confusion Matrix')
ax_lstm.set_xlabel('Predicted Category')
ax_lstm.set_ylabel('Actual Category')
ax_lstm.set_xticks(np.arange(len(labels_lstm)) + 0.5, labels=labels_lstm, rotation=45)
ax_lstm.set_yticks(np.arange(len(labels_lstm)) + 0.5, labels=labels_lstm)
plt.show()

# Calculate class-specific accuracies for the LSTM model
class_accuracies_lstm = cm_lstm.diagonal() / cm_lstm.sum(axis=1)
for i, acc in enumerate(class_accuracies_lstm):
    print(f'Accuracy for class {label_encoder_lstm.classes_[i]}: {acc:.2%}')

# Print classification report
print(classification_report(y_test_lstm, y_pred_classes_lstm, target_names=label_encoder_lstm.classes_))


# Example placeholders (replace these with actual values)

# Create a DataFrame to store results for LSTM

results_df_lstm = pd.DataFrame({
    'Accuracy': [accuracy_lstm],
    'Loss': [loss_lstm],
    'Confusion Matrix': [cm_lstm],
    'Class Accuracies': [class_accuracies_lstm],
    'Classification Report': [classification_report(y_test_lstm, y_pred_classes_lstm, target_names=label_encoder_lstm.classes_)]
})

# Save the DataFrame to an Excel file for LSTM
excel_file_name_lstm = "LSTM_results.xlsx"
results_df_lstm.to_excel(excel_file_name_lstm, index=False)
print(f"LSTM Results saved to {excel_file_name_lstm}")


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Plot the training and validation loss
loss = model_lstm.model_lstm['loss']
val_loss = model_lstm.model_lstm['val_loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss for CNN Model')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Save the plot as an image
plt.savefig('/content/CNN_results_Loss.png')

# Create a DataFrame with loss values
loss_data = pd.DataFrame({'Epochs': epochs, 'Training Loss': loss, 'Validation Loss': val_loss})

# Save the loss values to an Excel file
loss_data.to_excel('/content/CNN_results.xlsx', index=False)

# Display the plot
plt.show()

In [ ]:
# THis IS BILSTM USING DL   this is correct to used to DL correct accoracy

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.utils import class_weight
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Your data loading and merging code for BiLSTM
file_paths = ['/content/Sachal_jo_kalam_Scraping_data.xlsx', '/content/SHAH_jo_risalo_Scraping_data.xlsx', '/content/Ayaz_shaikh_Scraping_data.xlsx', '/content/Altaf_shaikh_Scraping_data.xlsx']

merged_data_bilstm = pd.DataFrame()

for file_path in file_paths:
    data = pd.read_excel(file_path)
    merged_data_bilstm = merged_data_bilstm.append(data, ignore_index=True)

# Check if the specified columns exist in the DataFrame
features_column_bilstm = 'Text Content'
target_column_bilstm = 'Category'
if set([features_column_bilstm, target_column_bilstm]).issubset(merged_data_bilstm.columns):
    features_bilstm = merged_data_bilstm[features_column_bilstm]
    target_bilstm = merged_data_bilstm[target_column_bilstm]

    label_encoder_bilstm = LabelEncoder()
    y_numerical_bilstm = label_encoder_bilstm.fit_transform(target_bilstm)

    tokenizer_bilstm = Tokenizer()
    tokenizer_bilstm.fit_on_texts(features_bilstm)
    total_words_bilstm = len(tokenizer_bilstm.word_index) + 1

    input_sequences_bilstm = tokenizer_bilstm.texts_to_sequences(features_bilstm)

    max_sequence_length_bilstm = max(len(seq) for seq in input_sequences_bilstm)
    padded_sequences_bilstm = pad_sequences(input_sequences_bilstm, maxlen=max_sequence_length_bilstm, padding='post')

    X_train_bilstm, X_test_bilstm, y_train_bilstm, y_test_bilstm = train_test_split(padded_sequences_bilstm, y_numerical_bilstm, test_size=0.2, random_state=42)

# Calculate class weights
class_weights_bilstm = compute_class_weight('balanced', classes=np.unique(y_train_bilstm), y=y_train_bilstm)
class_weights_dict_bilstm = dict(zip(np.unique(y_train_bilstm), class_weights_bilstm))

# Build and compile the BiLSTM model with class weights
learning_rate_bilstm = 0.00001  # You can adjust this value as needed
optimizer_bilstm = tf.keras.optimizers.Adam(learning_rate=learning_rate_bilstm)

model_bilstm = Sequential()
model_bilstm.add(Embedding(input_dim=total_words_bilstm, output_dim=32, input_length=max_sequence_length_bilstm))
model_bilstm.add(Bidirectional(LSTM(128)))  # Use Bidirectional layer with LSTM
model_bilstm.add(Dense(64, activation='relu'))
model_bilstm.add(Dense(len(label_encoder_bilstm.classes_), activation='softmax'))

model_bilstm.compile(optimizer=optimizer_bilstm, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the BiLSTM model with class weights
model_bilstm.fit(X_train_bilstm, y_train_bilstm, epochs=18, batch_size=32, validation_split=0.2, class_weight=class_weights_dict_bilstm)


# Save the trained CNN model
model_bilstm.save('spell_checker_BILSTM_model.h5')


# Evaluate the BiLSTM model and make predictions
loss_bilstm, accuracy_bilstm = model_bilstm.evaluate(X_test_bilstm, y_test_bilstm)
print(f'BiLSTM Model Test Loss: {loss_bilstm:.4f}, Test Accuracy: {accuracy_bilstm:.4f}')

# Make predictions on the test set using the BiLSTM model
y_pred_bilstm = model_bilstm.predict(X_test_bilstm)
y_pred_classes_bilstm = np.argmax(y_pred_bilstm, axis=1)

# Confusion matrix
cm_bilstm = confusion_matrix(y_test_bilstm, y_pred_classes_bilstm)
labels_bilstm = sorted(set(list(y_test_bilstm) + list(y_pred_classes_bilstm)))
plt.figure(figsize=(8, 6))
ax_bilstm = plt.gca()
sns.heatmap(cm_bilstm, annot=True, ax=ax_bilstm, cmap='Blues')
ax_bilstm.set_title('Confusion Matrix (BiLSTM)')
ax_bilstm.set_xlabel('Predicted Category')
ax_bilstm.set_ylabel('Actual Category')
ax_bilstm.set_xticks(np.arange(len(labels_bilstm)) + 0.5, labels=labels_bilstm, rotation=45)
ax_bilstm.set_yticks(np.arange(len(labels_bilstm)) + 0.5, labels=labels_bilstm)
plt.show()

# Calculate class-specific accuracies for the BiLSTM model
class_accuracies_bilstm = cm_bilstm.diagonal() / cm_bilstm.sum(axis=1)
for i, acc in enumerate(class_accuracies_bilstm):
    print(f'Accuracy for class {label_encoder_bilstm.classes_[i]}: {acc:.2%}')

# Print classification report for the BiLSTM model
print(classification_report(y_test_bilstm, y_pred_classes_bilstm, target_names=label_encoder_bilstm.classes_))


# Create a DataFrame to store results for BILSTM
results_df_bilstm = pd.DataFrame({
    'Accuracy': [accuracy_bilstm],
    'Loss': [loss_bilstm],
    'Confusion Matrix': [cm_bilstm],
    'Class Accuracies': [class_accuracies_bilstm],
    'Classification Report': [classification_report(y_test_bilstm, y_pred_classes_bilstm, target_names=label_encoder_bilstm.classes_)]
})

# Save the DataFrame to an Excel file for BILSTM
excel_file_name_bilstm = "BILSTM_results.xlsx"
results_df_bilstm.to_excel(excel_file_name_bilstm, index=False)
print(f"BILSTM Results saved to {excel_file_name_bilstm}")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Sample data (replace this with your actual data)
df = pd.read_excel('/content/merged_file.xlsx')

# Check if the DataFrame has data
if not df.empty:
    features_column = 'Text Content'
    target_column = 'Category'

    # Add a new column for line numbers
    df['Line Number'] = range(1, len(df) + 1)

    # Split the data into training and testing sets
    train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the training text data
    X_train = vectorizer.fit_transform(train_data[features_column])

    # Train a simple classifier (replace with your actual model)
    model = MultinomialNB()
    model.fit(X_train, train_data[target_column])

    # Evaluate the model on the test set (optional)
    X_test = vectorizer.transform(test_data[features_column])
    y_test = test_data[target_column]
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {accuracy * 100:.2f}%")

    # Function to predict the category of input text
    def predict_category(input_text, vectorizer, model):
        # Preprocess the input text
        input_features = vectorizer.transform([input_text])

        # Make the prediction
        predicted_category = model.predict(input_features)[0]

        return predicted_category

    # User input for searching text
    search_text = input("Enter the searching text: ")

    # Find the line number in the dataset
    line_number = df[df[features_column] == search_text]['Line Number'].values[0]

    # Make the prediction for the user input
    predicted_category = predict_category(search_text, vectorizer, model)

    print(f"The predicted category for the input text at line {line_number} is: {predicted_category}")
else:
    print("The DataFrame is empty.")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Sample data (replace this with your actual data)
try:
    df = pd.read_excel('/content/merged_file.xlsx')
    # Check if the required columns are present
    if 'Text Content' in df.columns and 'Category' in df.columns:
        features_column = 'Text Content'
        target_column = 'Category'

        # Add a new column for line numbers
        df['Line Number'] = range(1, len(df) + 1)

        # Split the data into training and testing sets
        train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

        # Create a TF-IDF vectorizer
        vectorizer = TfidfVectorizer()

        # Fit and transform the training text data
        X_train = vectorizer.fit_transform(train_data[features_column])

        # Train a simple classifier (replace with your actual model)
        model = MultinomialNB()
        model.fit(X_train, train_data[target_column])

        # Evaluate the model on the test set (optional)
        X_test = vectorizer.transform(test_data[features_column])
        y_test = test_data[target_column]
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Model Accuracy: {accuracy * 100:.2f}%")

        # Function to predict the category of input text
        def predict_category(input_text, vectorizer, model):
            # Preprocess the input text
            input_features = vectorizer.transform([input_text])

            # Make the prediction
            predicted_category = model.predict(input_features)[0]

            return predicted_category

        # User input for searching text
        search_text = input("Enter the searching text: ")

        # Check if the search_text exists in the DataFrame
        if search_text in df[features_column].values:
            # Find the line number in the dataset
            line_number = df[df[features_column] == search_text]['Line Number'].values[0]

            # Make the prediction for the user input
            predicted_category = predict_category(search_text, vectorizer, model)

            print(f"The predicted category for the input text at line {line_number} is: {predicted_category}")
        else:
            print("The entered text does not exist in the DataFrame.")
    else:
        print("Required columns (Text Content and Category) not found in the DataFrame.")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
# Words chacker  in all data set

import pandas as pd
import difflib
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('/content/spell_checker_CNN_model.h5')

# Load the Excel file containing the word list
word_list_df = pd.read_excel('/content/merged_file.xlsx')
word_list = word_list_df['Text Content'].tolist()

# Get user input
user_word = input("Enter a word: ")

# Check for misspelling
if user_word.lower() not in word_list:
  print("The word may be misspelled. Here are some suggestions:")

  # Find close matches using difflib
  close_matches = difflib.get_close_matches(user_word, word_list, n=5, cutoff=0.6)

  # Print suggestions, ensuring proper capitalization
  for suggestion in close_matches:
    if suggestion.istitle():
      print(suggestion)
    else:
      print(suggestion.capitalize())

else:
  print("The word is spelled correctly.")


In [ ]:
import pandas as pd
import difflib
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('/content/spell_checker_CNN_model.h5')

# Load the Excel file containing the word list
word_list_df = pd.read_excel('/content/merged_file.xlsx')
word_list = word_list_df['Text Content'].tolist()

# Get user input
user_word = input("Enter a sentence: ")

# Split the user input into words
user_words = user_word.split()

# Check each word for misspelling
for word in user_words:
    if word.lower() not in word_list:
        print("\033[91m" + word + "\033[0m", end=' ')  # Highlight misspelled word in red
    else:
        print(word, end=' ')

# Check for misspelled words
misspelled_words = [word for word in user_words if word.lower() not in word_list]

if misspelled_words:
    print("\n\nSome words may be misspelled. Here are suggestions:")

    # Provide suggestions for each misspelled word
    for misspelled_word in misspelled_words:
        # Find close matches using difflib
        close_matches = difflib.get_close_matches(misspelled_word, word_list, n=5, cutoff=0.6)

        # Print suggestions, ensuring proper capitalization
        suggestions = [suggestion if suggestion.istitle() else suggestion.capitalize() for suggestion in close_matches]
        print(f"{misspelled_word}: {', '.join(suggestions)}")
else:
    print("\nAll words are spelled correctly.")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

# Load the saved TensorFlow model
model = load_model('/content/spell_checker_BILSTM_model.h5')

# Get user input (assuming entire sentences or paragraphs)
user_input = input("Enter text to check: ")

# Split the input text into words, removing non-alphabetic characters
words = re.findall(r'\b\w+\b', user_input.lower())

# Tokenize the words and convert them to sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(words)
sequences = tokenizer.texts_to_sequences(words)

# Pad the sequences to have the same length (adjust maxlen based on model requirements)
maxlen = 38
padded_sequences = pad_sequences(sequences, maxlen=maxlen)

# Check each word using the model (assuming it generates suggestions)
for word, sequence in zip(words, padded_sequences):
    # Assuming the model outputs indices of suggestions
    sequence = sequence.reshape(1, -1)
    suggestions_indices = model.predict(sequence)

    # Convert indices to words using the tokenizer
    suggestions = [tokenizer.index_word[i] for i in suggestions_indices[0] if i in tokenizer.index_word]

    if not suggestions or word not in suggestions:
        print(f"'{word}' may be misspelled. Suggestions:")
        if suggestions:
            for suggestion in suggestions:
                print(suggestion.capitalize())
        else:
            print("No suggestions available.")
    else:
        print(f"'{word}' is spelled correctly.")


In [ ]:
import pandas as pd
import difflib
import tensorflow as tf
from tensorflow.keras.models import load_model
import gradio as gr
from transformers import pipeline

# Load the spell checking model and word list
model = load_model('/content/spell_checker_BILSTM_model.h5')
word_list_df = pd.read_excel('/content/merged_file.xlsx')
word_list = word_list_df['Text Content'].tolist()

# Load the sentiment analysis pipeline
sentiment_analysis = pipeline("sentiment-analysis")

def check_spelling_and_sentiment(text):
  """Checks the spelling of words in a text and analyzes its sentiment."""

  # Spell checking
  words = text.split()
  corrected_words = []
  for word in words:
    if word.lower() not in word_list:
      close_matches = difflib.get_close_matches(word, word_list, n=5, cutoff=0.6)
      suggestion = close_matches[0] if close_matches else word
      corrected_words.append(suggestion)
    else:
      corrected_words.append(word)

  corrected_text = " ".join(corrected_words)

  # Sentiment analysis
  sentiment = sentiment_analysis(corrected_text)[0]['label']

  return f"Corrected text: {corrected_text}\nSentiment: {sentiment}"

# Create the Gradio interface
iface = gr.Interface(
  fn=check_spelling_and_sentiment,
  inputs=[gr.Textbox(lines=5, placeholder="Enter a text to analyze")],
  outputs="text",
  title="Spell Checker and Sentiment Analysis",
  description="Checks spelling and analyzes sentiment",
)

# Launch the interface
iface.launch()
